In [80]:
import boto3
import os

In [81]:
#variables
OHIO_REGION = "us-east-2"
NORTH_VIRGINIA = "us-east-1"
AMI_UBUNTU_OHIO = "ami-020db2c14939a8efb"
AMI_UBUNTU_NORTH_VIRGINIA = "ami-0279c3b3186e54acd"
KEY_PAIR_OHIO_NAME = "key_pair_marcelo_ohio"
KEY_PAIR_NORTH_VIRGINIA_NAME = "key_pair_marcelo_north_virginia"
SEC_GROUP_NAME_DB = "Sec_Group_DB"
SEC_GROUP_NAME_DJ = "Sec_Group_DJ"
IMAGE_DJ = "Image Django Instance"
LB_NAME = "LB-Django"

In [82]:
# commands line
comand_db="""
#cloud-config

runcmd:
- cd /
- sudo apt update
- sudo apt install postgresql postgresql-contrib -y
- sudo su - postgres
- sudo -u postgres psql -c "CREATE USER cloud WITH PASSWORD 'cloud';"
- sudo -u postgres psql -c "CREATE DATABASE tasks;"
- sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE tasks TO cloud;"
- sudo echo "listen_addresses = '*'" >> /etc/postgresql/10/main/postgresql.conf
- sudo echo "host all all 0.0.0.0/0 trust" >> /etc/postgresql/10/main/pg_hba.conf
- sudo ufw allow 5432/tcp -y
- sudo systemctl restart postgresql

"""

command_django="""
#cloud-config

runcmd:
- cd /home/ubuntu 
- sudo apt update -y
- git clone https://github.com/MarceloCMiguel/tasks.git
- cd tasks
- sed -i "s/node1/IP_REPLACE/g" ./portfolio/settings.py
- ./install.sh
- sudo ufw allow 8080/tcp -y
- sudo reboot
"""

In [83]:
# permissions
IpPermissions_DB = [
    {
        'IpProtocol': 'tcp',
        'FromPort': 22,
        'ToPort': 22,
        'IpRanges': [
            {'CidrIp': '0.0.0.0/0'}
        ]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 5432,
        'ToPort': 5432,
        'IpRanges': [
            {'CidrIp': '0.0.0.0/0'}
        ]
    },
    {
            "FromPort": 80,
            "ToPort": 80,
            "IpProtocol": "tcp",
            "IpRanges": [
                {"CidrIp": "0.0.0.0/0", "Description": "internet"},
            ],
        },
]


IpPermissions_DJ = [
    {
        'IpProtocol': 'tcp',
        'FromPort': 22,
        'ToPort': 22,
        'IpRanges': [
            {'CidrIp': '0.0.0.0/0'}
        ]
    },
    {
        'IpProtocol': 'tcp',
        'FromPort': 8080,
        'ToPort': 8080,
        'IpRanges': [
            {'CidrIp': '0.0.0.0/0'}
        ]
    },
    {
            "FromPort": 80,
            "ToPort": 80,
            "IpProtocol": "tcp",
            "IpRanges": [
                {"CidrIp": "0.0.0.0/0", "Description": "internet"},
            ],
        },
]


In [153]:
#create a key_pair
def create_key_pair(client,key_name:str):
    key_pair = client.create_key_pair(KeyName=key_name)

    private_key = key_pair["KeyMaterial"]

    # write private key to file with 400 permissions
    print(f"Writing {key_name}.pem in directory ./")
    with os.fdopen(os.open(f"./{key_name}.pem", os.O_WRONLY | os.O_CREAT, 0o400), "w+") as handle:
        handle.write(private_key)
    return key_pair

#delete a key_pair
def delete_key_pair(client,key_name:str):
    key_pair = client.delete_key_pair(KeyName=key_name)
    print(f"removing {key_name} if exist")
    # write private key to file with 400 permissions
    try:
        os.remove("./" + key_name + ".pem")
        
    except:
        pass


def create_an_instance(client, ami,key_name,user_data,sec_group_name,sec_group_id):
    
    instancetype = "t2.micro"
    print(f"Creating an instance with {instancetype}, and keyname {key_name['KeyName']}")
    if user_data == None:
        instances = client.run_instances(
            ImageId=ami,
            MinCount=1,
            MaxCount=1,
            InstanceType="t2.micro",
            KeyName=key_name['KeyName'],
            SecurityGroupIds=[sec_group_id],
            SecurityGroups=[sec_group_name]
        )
    else:
        instances = client.run_instances(
            ImageId=ami,
            MinCount=1,
            MaxCount=1,
            InstanceType="t2.micro",
            KeyName=key_name['KeyName'],
            UserData = user_data,
            SecurityGroupIds=[sec_group_id],
            SecurityGroups=[sec_group_name]
        )
    for i in instances['Instances']:
        if i['KeyName'] == key_name["KeyName"]:
            instance_id = i['InstanceId']
    waiter = client.get_waiter('instance_status_ok')
    waiter.wait(InstanceIds=[instance_id])
    instance_ip = client.describe_instances(Filters=[
        {
            'Name': 'key-name',
            'Values': [
                key_name['KeyName'],
            ]
        },
        {
            'Name': 'instance-state-name',
            'Values': [
                "running"
            ]
        },
    ],
    )['Reservations'][0]['Instances'][0]['PublicIpAddress']
    print(f"Instance id: {instance_id}\nInstance Public IP:{instance_ip}")
    return instance_id, instance_ip

def delete_instances(client,key_pair):
    list_instances_id = []
    for i in client.describe_instances(Filters=[
        {
            'Name': 'key-name',
            'Values': [
                key_pair,
            ]
        },
        {
            'Name': 'instance-state-name',
            'Values': [
                "pending","running","stopping","stopped"
            ]
        },
    ],
)['Reservations']:
        list_instances_id.append(i['Instances'][0]["InstanceId"])
    if len(list_instances_id) > 0:
        print(f"Deleting Instances ids {list_instances_id}")
        waiter = client.get_waiter('instance_terminated')
        client.terminate_instances(InstanceIds=list_instances_id)
        waiter.wait(InstanceIds=list_instances_id)
        print(f"Instances withs keypair {key_pair} deleted")
    else:
        print(f"There is no instances running with the keypair {key_pair}")
        
def delete_sec_group(client,sec_group_name):
    find_sec_group = False
    security_groups = client.describe_security_groups()
    for sec_group in security_groups['SecurityGroups']:
        if sec_group['GroupName'] == sec_group_name:
            find_sec_group = True
            
            deleted_sec_group = client.delete_security_group(GroupName=sec_group["GroupName"], GroupId=sec_group["GroupId"])
    if find_sec_group:
        print(f"{sec_group_name} deleted")
    else:
        print(f"{sec_group_name} not found")
def create_security_group(client, SEC_GROUP_NAME,IP_PERMISSIONS):
    vpcs = client.describe_vpcs()
    vpc_id = vpcs['Vpcs'][0]['VpcId']
    security_group = client.create_security_group(GroupName=SEC_GROUP_NAME,
                                                Description="Created with boto3",
                                                VpcId=vpc_id)
    security_group_id = security_group['GroupId']
    client.authorize_security_group_ingress(
        GroupId=security_group_id,
        IpPermissions = IP_PERMISSIONS
    )
    print(f"Security Group {SEC_GROUP_NAME} created")
    return security_group_id

def create_image(client,instance_id,name_image):
    image_response=client.create_image(InstanceId=instance_id,Name=name_image)
    waiter = client.get_waiter('image_available')
    image_id = image_response["ImageId"]
    waiter.wait(ImageIds=[image_id])
    print(f"Image {name_image} created with id {image_id}")
    return image_id
    
def delete_image(client,image_name):
    images_described=ec2_north_virginia_.describe_images(Filters=[
        {
            'Name': 'name',
            'Values': [
                image_name,
            ]
        },
    ])
    if len(images_described['Images'])<1:
        print(f"There is no image with name {image_name}")
        return
    image_id = images_described['Images'][0]['ImageId']
    client.deregister_image(ImageId=image_id)
    print(f"Image {image_name} deleted")
    
    
def create_load_balancer(client,client_lb, lb_name,sec_group_id):    
    subnets = client.describe_subnets()
    list_subnets_id = []
    for subnet in subnets['Subnets']:
        list_subnets_id.append(subnet['SubnetId'])
    waiter = client_lb.get_waiter('load_balancer_available')    
    print("Creating load balancer with all subnets available")
    load_balancer_created = client_lb.create_load_balancer(Name = lb_name,
                                                                    Subnets=list_subnets_id,
                                                                    SecurityGroups=[sec_group_id],
                                                                    IpAddressType="ipv4",)
    LoadBalancerArn_ = load_balancer_created['LoadBalancers'][0]['LoadBalancerArn']
    waiter.wait(LoadBalancerArns=[LoadBalancerArn_])
    print(f"Load Balancer {lb_name} created")
    return LoadBalancerArn_

def delete_load_balancer(client_lb,lb_name):
    waiter = client_lb.get_waiter('load_balancers_deleted')
    try:
        load_balancers = client_lb.describe_load_balancers(Names=[lb_name])
    except:
        print(f"No load_balancers with name {lb_name}")
        return
    for lb in load_balancers['LoadBalancers']:
        client_lb.delete_load_balancer(LoadBalancerArn=lb['LoadBalancerArn'])
    print(f"Load Balancer with name {lb_name} deleted")

In [85]:
#checking if boto3 client is working
iam = boto3.client("iam")

for user in iam.list_users()["Users"]:
    if user["UserName"] == "marcelo":
        print(user["UserName"])
        print(user["UserId"])
        print(user["Arn"])
        print(user["CreateDate"])

marcelo
AIDATTPQFU3W27CNBJLEK
arn:aws:iam::248001177325:user/marcelo
2021-09-21 18:22:34+00:00


In [154]:
#connection to ec2
ec2_ohio = boto3.client('ec2', region_name='us-east-2')
ec2_north_virginia_ = boto3.client('ec2', region_name='us-east-1')
lb_north_virginia_ = boto3.client('elbv2',region_name="us-east-1")
as_north_virginia_ = boto3.client('autoscaling',region_name="us-east-1")

In [87]:
delete_image(ec2_north_virginia_,IMAGE_DJ)

There is no image with name Image Django Instance


In [ ]:
delete_load_balancer(lb_north_virginia_,LB_NAME)

In [88]:
delete_instances(ec2_ohio,KEY_PAIR_OHIO_NAME)
delete_instances(ec2_north_virginia_,KEY_PAIR_NORTH_VIRGINIA_NAME)

There is no instances running with the keypair key_pair_marcelo_ohio
There is no instances running with the keypair key_pair_marcelo_north_virginia


In [89]:
delete_key_pair(ec2_ohio,KEY_PAIR_OHIO_NAME)
delete_key_pair(ec2_north_virginia_,KEY_PAIR_NORTH_VIRGINIA_NAME)

removing key_pair_marcelo_ohio if exist
removing key_pair_marcelo_north_virginia if exist


In [90]:
delete_sec_group(ec2_ohio,SEC_GROUP_NAME_DB)
delete_sec_group(ec2_north_virginia_,SEC_GROUP_NAME_DJ)

Sec_Group_DB not found
Sec_Group_DJ not found


In [91]:
sec_group_id_db = create_security_group(ec2_ohio,SEC_GROUP_NAME_DB,IpPermissions_DB)
sec_group_id_dj = create_security_group(ec2_north_virginia_,SEC_GROUP_NAME_DJ,IpPermissions_DJ)

Security Group Sec_Group_DB created
Security Group Sec_Group_DJ created


In [92]:
key_pair_ohio = create_key_pair(ec2_ohio,KEY_PAIR_OHIO_NAME)
key_pair_north_virginia = create_key_pair(ec2_north_virginia_,KEY_PAIR_NORTH_VIRGINIA_NAME)

Writing key_pair_marcelo_ohio.pem in directory ./
Writing key_pair_marcelo_north_virginia.pem in directory ./


In [93]:
postgres_ID, postgres_IP = create_an_instance(ec2_ohio,AMI_UBUNTU_OHIO,key_pair_ohio,comand_db,SEC_GROUP_NAME_DB,sec_group_id_db)
command_django = command_django.replace("IP_REPLACE",str(postgres_IP))
django_ID, django_IP = create_an_instance(ec2_north_virginia_,AMI_UBUNTU_NORTH_VIRGINIA,key_pair_north_virginia,command_django,SEC_GROUP_NAME_DJ,sec_group_id_dj)

Creating an instance with t2.micro, and keyname key_pair_marcelo_ohio
Instance id: i-0830ffe54dad3bea2
Instance Public IP:3.143.213.44
Creating an instance with t2.micro, and keyname key_pair_marcelo_north_virginia
Instance id: i-00fdc278cf493ad0b
Instance Public IP:3.94.166.111


In [94]:
image_dj = create_image(ec2_north_virginia_,django_ID,IMAGE_DJ)
delete_instances(ec2_north_virginia_,KEY_PAIR_NORTH_VIRGINIA_NAME)

Image Image Django Instance created with id ami-09babbc21c5a3c49a
Deleting Instances ids ['i-00fdc278cf493ad0b']
Instances withs keypair key_pair_marcelo_north_virginia deleted


In [149]:
load_balancer_arn = create_load_balancer(ec2_north_virginia_,lb_north_virginia_,LB_NAME,sec_group_id_dj)